In [1]:
%autosave 10
import time
import os
import pandas as pd

Autosaving every 10 seconds


In [2]:
#Size of the file in bytes
os.path.getsize('Parking_Violations_Issued_-_Fiscal_Year_2016.csv')

2151937808

In [3]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
end = time.time()
print("Reading csv with dask takes: ",(end-start),"sec")

Reading csv with dask takes:  0.46426892280578613 sec


In [4]:
#import using dask
from dask import dataframe as dd
df = dd.read_csv('Parking_Violations_Issued_-_Fiscal_Year_2016.csv',delimiter=',')

In [5]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 51 entries, Summons Number to NTA
dtypes: object(21), float64(16), int64(14)

In [ ]:
#No of Rows
import pandas as pd
df = pd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
print(df)
index = (df.index)
number_of_rows = len(index)
print(number_of_rows)

C:\Users\Shiwani\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,34,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
#No of Columns
len(df.columns)

In [ ]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [ ]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [ ]:
data=df.columns
data

In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
   
    
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [ ]:
%%writefile file.yaml
file_type: csv
dataset_name: Parking_Violations_Issued_-_Fiscal_Year_2016
file_name: Parking_Violations_Issued_-_Fiscal_Year_2016
table_name: Parking_Violations_Issued_-_Fiscal_Year_2016
inbound_delimiter: ","
outbound_delimiter: ","
skip_leading_rows: 1
columns:
    - SummonsNumber 
    - PlateID 
    - RegistrationState
    - PlateType
    - IssueDate
    - ViolationCode
    - VehicleBodyType 
    - VehicleMake
    - IssuingAgency
    - StreetCode1
    - StreetCode2 
    - StreetCode3
    - VehicleExpirationDate 
    - ViolationLocation 
    - ViolationPrecinct
    - IssuerPrecinct 
    - IssuerCode
    - IssuerCommand
    - IssuerSquad
    - ViolationTime 
    - TimeFirstObserved 
    - ViolationCounty
    - ViolationInFrontOfOrOpposite 
    - HouseNumber 
    - StreetName
    - IntersectingStreet
    - DateFirstObserved 
    - LawSection
    - SubDivision
    - ViolationLegalCode 
    - DaysParkingInEffect 
    - FromHoursInEffect
    - ToHoursInEffect
    - VehicleColor 
    - UnregisteredVehicle
    - VehicleYear 
    - MeterNumber
    - FeetFromCurb
    - ViolationPostCode
    - ViolationDescription
    - NoStandingorStoppingViolation
    - HydrantViolation
    - DoubleParkingViolation
    - Latitude 
    - Longitude
    - CommunityBoard 
    - CommunityCouncil 
    - CensusTract
    - BIN 
    - BBL
    - NTA

In [ ]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [ ]:
#inspecting data of config file
config_data

In [ ]:
import pandas as pd

In [ ]:
file_type= config_data['file_type']
source_file= "./" + config_data['file_name'] + f'.{file_type}'

df= pd.read_csv(source_file, config_data['inbound_delimiter'])

In [ ]:
util.col_header_val(df, config_data)

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

In [ ]:
if util.col_header_val(df, config_data)==0:
    print("validation is failed")
    
else:
    print("validation is passed")

In [ ]:
mport datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2016.csv",delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('Parking_Violations_Issued_-_Fiscal_Year_2016.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

In [ ]:
#size of the gz format folder in bytes
os.path.getsize('Parking_Violations_Issued_-_Fiscal_Year_2016.csv.gz')

In [ ]:
df.info()

In [ ]:
data=df.columns
data

In [ ]:
#No, of Columns
len(df.columns)

In [ ]:
#No. of Rows
len(df.index)